# 1. Import the necessary libraries for you to be able to query 4square API

In [69]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
from dotenv import load_dotenv
load_dotenv()
import time

In [70]:
token = os.getenv("token")

In [82]:
def gets_coordinates_from_string (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    
    try:
        return res.json()
    
    except:
        time.sleep(2)
        gets_coordinates_from_string (where)

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [93]:
res = gets_coordinates_from_string ("Calle de Pamplona, 96, 08018 Barcelona, Spain")
res

{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.8'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [94]:
lat, lon = res["latt"], res["longt"]
lat

'41.39780'

In [160]:
def requests_for_foursquare (query, lat, lon, radius=700, limit=1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [161]:
res = requests_for_foursquare ("coffee", lat, lon, radius=600, limit=10)
len(res)

2

In [119]:
def extracting_from_one_element (searched_place):

    name = searched_place["name"]
    address = searched_place["location"]["address"]
    distance = searched_place["distance"]
    #zip_code = searched_place["location"]["postcode"]
    lat = searched_place["geocodes"]["main"]["latitude"]
    lon = searched_place["geocodes"]["main"]["longitude"]

    
    small_dict = {
    "name": name,
    "address": address,
    #"zip_code": zip_code,
    "distance": distance,
    "lat": lat,
    "lon": lon
}
    return small_dict

In [120]:
def transform_request_into_df (res):
    list_of_dictionaries = [extracting_from_one_element (element) for element in res["results"]]
    return pd.DataFrame(list_of_dictionaries)

In [121]:
df =transform_request_into_df (res)
df

,name,address,distance,lat,lon
0,SKYE Coffee Co,"Calle Pamplona, 88",135,41.396937,2.191519
1,Syra Coffee Poblenou,"Calle de Pujades, 100",342,41.396518,2.194170
2,Espai Joliu,"Carrer Badajoz, 95",399,41.398735,2.195069
3,Nømad Roaster's Home,"Calle de Pujades, 95",450,41.397977,2.195589
4,Eva-2,"Bolivia, 21",439,41.401636,2.189006
5,Mare Meva,"C. de Bolívia, 31",487,41.402167,2.189879
6,Cafeteria Bar BAU,"Pujades, 118",493,41.397524,2.196263
7,El Bar,"Ribes, 46",572,41.398897,2.183774
8,Espacio 88,"Carrer Pamplona, 88",124,41.396993,2.191171
9,Granja Mabel,"Marina, 114-116",418,41.394816,2.187354


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [122]:
#as we have the distance, we just need to sort by distance ascending and then get the head(5)

closest_5 = df.sort_values('distance').head(5)
closest_5

,name,address,distance,lat,lon
8,Espacio 88,"Carrer Pamplona, 88",124,41.396993,2.191171
0,SKYE Coffee Co,"Calle Pamplona, 88",135,41.396937,2.191519
1,Syra Coffee Poblenou,"Calle de Pujades, 100",342,41.396518,2.194170
2,Espai Joliu,"Carrer Badajoz, 95",399,41.398735,2.195069
9,Granja Mabel,"Marina, 114-116",418,41.394816,2.187354


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [224]:
# as nothing indicated I have fixed radius = 500m and limit the anwers to 5. 

def requests_for_foursquare (query, lat, lon, radius=700, limit=2):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        f4_answer = requests.get(url, headers=headers).json()
    except:
        print("no :(")
    
    answer_list = []
    answer = f4_answer["results"]
    
    for element in answer:
        
        name = element["name"]
        address = element["location"]["address"]
        distance = element["distance"]
        lat = element["geocodes"]["main"]["latitude"]
        lon = element["geocodes"]["main"]["longitude"]
        
        small_dict = {
        "name": name,
        "address": address,
        "distance": distance,
        "lat": lat,
        "lon": lon}
        
        answer_list.append(small_dict)
        
    
    df=pd.DataFrame(answer_list)
    return df



#list_of_dictionaries = [extracting_from_one_element (element) for element in res["results"]]

In [225]:
lon

'2.19035'

In [226]:
df = requests_for_foursquare ("coffee", lat, lon, radius=600, limit=10)

In [223]:
df

,name,address,distance,lat,lon
0,SKYE Coffee Co,"Calle Pamplona, 88",135,41.396937,2.191519
1,Syra Coffee Poblenou,"Calle de Pujades, 100",342,41.396518,2.194170
2,Espai Joliu,"Carrer Badajoz, 95",399,41.398735,2.195069
3,Nømad Roaster's Home,"Calle de Pujades, 95",450,41.397977,2.195589
4,Eva-2,"Bolivia, 21",439,41.401636,2.189006
5,Mare Meva,"C. de Bolívia, 31",487,41.402167,2.189879
6,Cafeteria Bar BAU,"Pujades, 118",493,41.397524,2.196263
7,El Bar,"Ribes, 46",572,41.398897,2.183774
8,Espacio 88,"Carrer Pamplona, 88",124,41.396993,2.191171
9,Granja Mabel,"Marina, 114-116",418,41.394816,2.187354


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [119]:

closest_5.to_json('closest_5.json', orient='records', lines=True)

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [116]:
pwd

'C:\\PATRICIA\\Ironhack\\Laboratories\\week-4\\lab-coffee-shops'